In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import sys
import os
# 添加路径到系统路径
sys.path.append("/content/drive/MyDrive/transformer")
# Change working directory to the script's location
os.chdir("/content/drive/MyDrive/transformer")

# raw data
path_do_data = "data/english.txt"
data_raw = open(path_do_data, encoding="utf-8").read()

In [ ]:
!pip install -r /content/drive/MyDrive/transformer/requirements.txt


In [ ]:
!python /content/drive/MyDrive/transformer/train_model.py

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Token indices sequence length is longer than the specified maximum sequence length for this model (1239778 > 512). Running this sequence through the model will result in indexing errors
Model with 89.48M parameters
Training Progress:   0% 0/5000 [00:00<?, ?it/s]step          0 | train loss 10.6753 | val loss 10.6958
Training Progress:   1% 63/5000 [21:19<27:50:43, 20.30s/it]
Traceback (most recent call last):
  File "/content/drive/MyDrive/transformer/train_model.py", line 80, in <module>
    logits, loss = m.forward(xb, yb)
  File "/content/drive/MyDrive/transformer/model.py", line 169, in forward
    

In [ ]:
# 加载并使用训练好的模型进行推理
import torch
from transformers import AutoTokenizer
from model import Transformer
from utils import load_model_from_checkpoint, decode

In [ ]:
# 设置参数
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
path_to_checkpoint = "/content/drive/MyDrive/transformer/checkpoints/state_dict_model.pt"
vocab_path = "bert-base-uncased"

# 加载分词器
tokenizer = AutoTokenizer.from_pretrained(vocab_path)
vocab_size = tokenizer.vocab_size

# 加载模型
model = Transformer(
    vocab_size=vocab_size,
    num_embed=768,  # 假设 num_embed=768，与之前训练时一致
    block_size=64,
    num_heads=6,
    num_layers=6,
    dropout=0.2,
).to(DEVICE)

# hyperparameters
BATCH_SIZE = 32  # how many independent sequences will we process in parallel?
BLOCK_SIZE = 64  # what is the maximum context length for predictions?
MAX_ITER = 5000  # number of training iterations
EVAL_INTER = 500
LEARNING_RATE = 3e-4
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
NUM_HEAD = 6
NUM_EMBED = NUM_HEAD * 128
NUM_LAYER = 6
DROPOUT = 0.2

# 从检查点加载权重
model = load_model_from_checkpoint(model_class=Transformer, path_to_checkpoint=path_to_checkpoint, vocab_size=vocab_size)

In [ ]:
# 生成文本
context = torch.zeros((1, 1), dtype=torch.long, device=DEVICE)
generated_tokens = model.generate(idx=context, max_new_tokens=100, block_size=64)[0]
generated_text = decode(enc_sec=generated_tokens, tokenizer=tokenizer)

print("Generated text:\n", generated_text)